# Discrete and Fast Fourier Transform

by Felix Fritzen fritzen@mib.uni-stuttgart.de, November 2020

additional material for the course *Data Processing for Engineers and Scientists*

## Content
- FFT/DFT for convolution in Fourier space (simplistic 1D example)
- FFT/DFT of harmonic signals
- 2D convolution used for ASCII art


In [5]:
import numpy as np
from numpy.fft import fft, ifft, fft2, ifft2
from scipy.linalg import dft


## 1D simplistic example

We start from a simple signal
$$ \underline{a} = [ 0, 1, 2, 3, 2, 1, 0, \dots, 0 ]^\mathsf{T} \in \mathbb{R}^{20} $$

A second vector is
$$ \underline{b} \in \mathbb{R}^{20}: b_i = \left\lbrace\begin{array}{cc} 1 & i \in \{ 1, 7 \}\\ 0 & \text{else} \end{array}\right.$$

In [6]:
a = np.zeros(20)
a[np.arange(1,4)] = np.arange(1,4)
a[np.arange(1,3)+3] = 3-np.arange(1,3)
b = np.zeros(20)
b[0] = 1
b[6] = 1
f_a = fft(a)
f_b = fft(b)
f_ab = f_a*f_b
ab = ifft(f_ab)
print('original signal - a:')
print(a)
print('second signal   - b:')
print(b)
print('convolution     - a*b:')
print(np.round(np.real(ab)))
c = np.zeros(20)
c[0] = 1
c[3] = 1
ac = np.round(np.real(ifft( f_a * fft(c) )))
print('third signal     - c:')
print(c)
print('convolution     - a*c:')
print(ac)


original signal - a:
[0. 1. 2. 3. 2. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
second signal   - b:
[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
convolution     - a*b:
[-0.  1.  2.  3.  2.  1.  0.  1.  2.  3.  2.  1.  0.  0.  0.  0.  0.  0.
  0.  0.]
third signal     - c:
[1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
convolution     - a*c:
[ 0.  1.  2.  3.  3.  3.  3.  2.  1. -0. -0. -0. -0. -0. -0. -0.  0.  0.
  0.  0.]



### observations
- convolution of $\underline{a}$ by a zero vector with only non-zero entry $x_j\in\mathbb{R}\setminus \{0\}$ is equivalent to shifting $\underline{a}$ by $j$ ($j$ assumed to be zero based) and weighting the signal by $x_j$
- the convolution $a \ast b$ replicates the signal starting at positions 0 and 6 respectively; the two translated signals don't overlap
- the convolution $a \ast c$ replicates the signal starting at positions 0 and 3 respectively; the two translated signals overlap $\to$ values are summed up

**major application in the sequel: signal and image filtering**

## 1D FFT/DFT of harmonic functions

Let $\underline{x} \in \mathbb{R}^n$ a uniform grid on the interval $[0, (1-1/n)]$.

We define a second vector $\underline{y}\in\mathbb{R}^n$ using a phase shift $\delta\in [0,2\pi]$ via
$$ y_i = \sin( 2\pi \, k x_i + \delta ) \qquad (i=1,\dots,n; k \in \{ 1, \dots, \lfloor n/2 \rfloor \} ) $$

### analyze the vector $\underline{y}$ using FFT

In [37]:
n = 128
x = np.arange(0,n)/n
k = 16
delta = x[76]*2*np.pi
#y = np.cos(2*np.pi*k*x + delta)
y = np.sin(2*np.pi*k*x + delta)

f_y = fft(y)/n # ATTENTION: divide by n
#print(np.abs(f_y))
print('y                = sin(2pi*x*k + delta); k=%d; delta=%12.8f' % (k, delta))
print('F(y)_k           = %8.4f + %8.4fi' % (np.real(f_y[k]), np.imag(f_y[k])))
print('F(y)_n-k         = %8.4f + %8.4fi' % (np.real(f_y[-k]), np.imag(f_y[-k])))
print('0.5*cos(delta)   = %8.4f' % (0.5*np.cos(delta)))
print('0.5*sin(delta)   = %8.4f' % (0.5*np.sin(delta)))
# phase shift for cosine function
delta_comp = np.mod(np.arctan2(np.imag(f_y[k]), np.real(f_y[k])), 2*np.pi)
# phase shift for sine function --> add pi/2
print('determined delta (asserting sine):   %12.8f vs. %12.8f' % ( np.mod( delta_comp+np.pi/2, 2*np.pi) , delta))
x=np.random.normal(size=100)


y                = sin(2pi*x*k + delta); k=16; delta=  3.73064128
F(y)_k           =  -0.2778 +   0.4157i
F(y)_n-k         =  -0.2778 +  -0.4157i
0.5*cos(delta)   =  -0.4157
0.5*sin(delta)   =  -0.2778
determined delta (asserting sine):     3.73064128 vs.   3.73064128


### observations
- note that `numpy.fft` does not provide the division by $n$ (see code snippet above)
- the absolute value of $\widehat{y}_k$ and $\widehat{y}_{n-k}=\text{conj}(\widehat{y}_k)$ ($k$ zero based) is the amplitude of an oscillation at frequency $k$ in the signal
- the phase shift $\delta$ can be computed from the ratio of imaginary and real value of $\widehat{y}_k$:
$$ \delta = \text{atan2} \left( \text{imag}\left(\widehat{y}_k\right), \text{real}\left(\widehat{y}_k\right) \right)$$
- the previous statements can be exploited to analyze the frequency spectrum of a signal
- filtering the signal in Fourier space will result into filtering the frequency content (e.g. for use in an equalizer)


---
## 2D FFT for making ASCII art


In [4]:
candle = np.zeros((8,32),dtype='int')

candle[:,:8] = np.array([[0,0,0,0,1,0,0,0],
[0,0,0,1,8,1,0,0],
[0,0,0,0,1,0,0,0],
[0,0,8,8,8,8,8,0],
[0,0,1,8,8,8,1,0],
[0,0,1,8,8,8,1,0],
[0,0,1,8,8,8,1,0],
[7,7,7,7,7,7,7,7]])
print('Candle prototype:')
print(candle)
print()
print('improved visualization:')
print(str(candle).replace(" ", "").replace("[","").replace("]","").replace("0"," "))


f_candle = fft2(candle)

advent_first=np.zeros((8,32))
advent_first[0][0] = 1
advent_second=advent_first.copy()
advent_second[0][8] = 1
advent_third=advent_second.copy()
advent_third[0][16] = 1
advent_fourth=advent_third.copy()
advent_fourth[0][24] = 1

advent_first = np.round(np.real(ifft2( fft2(advent_first)  * f_candle ) )).astype(int)
advent_second= np.round(np.real(ifft2( fft2(advent_second) * f_candle ) )).astype(int)
advent_third = np.round(np.real(ifft2( fft2(advent_third)  * f_candle ) )).astype(int)
advent_fourth= np.round(np.real(ifft2( fft2(advent_fourth) * f_candle ) )).astype(int)
print()
print('First Sunday of Advent, Nov 29, 2020:')
print(str(advent_first).replace(" ", "").replace("[","").replace("]","").replace("0"," "))

print()
print('Second Sunday of Advent, Dec 6, 2020:')
print(str(advent_second).replace(" ", "").replace("[","").replace("]","").replace("0"," "))

print()
print('Third Sunday of Advent, Dec 13, 2020:')
print(str(advent_third).replace(" ", "").replace("[","").replace("]","").replace("0"," "))

print()
print('Fourth Sunday of Advent, Dec 20, 2020:')
print(str(advent_fourth).replace(" ", "").replace("[","").replace("]","").replace("0"," "))


Candle prototype:
[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 8 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 8 8 8 8 8 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 8 8 8 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 8 8 8 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 8 8 8 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [7 7 7 7 7 7 7 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

improved visualization:
    1                           
   181                          
    1                           
  88888                         
  18881                         
  18881                         
  18881                         
77777777                        

First Sunday of Advent, Nov 29, 2020:
    1                           
   181                          
    1                           
  88888           